# Импорты

In [ ]:
import subprocess
import sys
import json

import pandas as pd
from matplotlib import pyplot as plt

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    subprocess.run(["pip", "install", "catboost>=1.2.7"])

# Для более качественных графиков
%config InlineBackend.figure_format='retina'
plt.rcParams["figure.dpi"] = 150

# Форматы данных: csv, json

## Csv

In [ ]:
pd.read_csv("data/housing.csv")

## Json

In [ ]:
with open("data/addres-book.json", "r") as f:
    lines = f.readlines()

lines

In [ ]:
with open("data/addres-book.json", "r") as f:
    json_data = json.load(f)

json_data

Задача: соберите все уникальные телефоны в этом json

## Работа с файловой системой

In [ ]:
from pathlib import Path

p = Path()  # очень удобный способ взаимодействовать с файловой системой

# Обогащение, разметка датасетов

## Feature engineering

Для улучшения качества моделей можно придумывать новые признаки на основании уже существующих

Способы создания новых признаков:
- Математические операции (возведение в квадрат, логарифм)
- На основе условий (больше ли этот признак какого-то значения, является ли он максимально возможным)
- Комбинация нескольких признаков
- Агрегация (среднее этого признака, сколько раз в тексте встречалось какое-то слово)

In [ ]:
df = pd.read_csv("data/train.csv")
df["речь"]

In [ ]:
df

## Регулярные выражения

Регулярные выражение - формальный язык для составления текстовых шаблонов. При помощи этих шаблонов можно искать и обрабатывать текст

Создать откомпилированный шаблон регулярного выражения позволяет функция `compile()`

In [ ]:
import re

# Шаблон, соответствующий строке, начинающейся на 0-6 русских букв
p = re.compile(r"[а-яё]{0,6}")
# Найдем в строке "тиквввввtik" подстроку, которая удовлетворяет шаблону
p.match("тиквввввtik").group(0)

__Специальные символы регулярных выражений__

Внутри регулярного выражения символы `.   ^   $  *  +  ?   {  } [  ]  \  |  ( ) -` имеют специальное значение. 
* Если эти символы требуется выводить как есть, то их следует экранировать с помощью слэша `\`.
* Некоторые сnециальные символы теряют свое особое значение, если их разместить внутри квадратных скобок `[]`. В этом случае экранировать их не нужно. 

__Основные методы__

* `re.match()` - Этот метод ищет по заданному шаблону в начале строки. Возвращает первое вхождение подстроки в виде объекта SRE_Match object, из которого:
    * можно получить результирующую подстроку с помощью функции group
    * индексы начальной и конечной позиции с помощью функций start() и end(), соответственно.
* `re.search()` - ищет по заданному шаблону во всей строке
* `re.findall()` - возвращает список всех найденных совпадений (подстрок).
* `re.split()` - разделяет строку по заданному шаблону
* `re.sub()` - заменяет шаблон на указанную подстроку.

В квадратных скобках `[]` можно указать символы, которые могут встречаться на этом месте в строке. Можно перечислять символы подряд или указать их диапазон через тире. Например:
* `[09]` - соответствует цифре 0 или 9 <br>
* `[0-9]` - соответствует одной цифре от 0 до 9 <br>
* `[абв]` - соответствует букве "а", "б" или "в" <br>
* `[а-г]` - соответствует букве "а", "б", "в" или "г" <br>
* `[а-я]` - соответствует любой букве от "а" до "я", кроме буквы "ё" (т.к. "ё" находится вне непрерывного дипаозона символов русского алфавита) <br>
* `[а-яё]` - соответствует любой букве от "а" до "я" <br>
* `[АБВ]` - соответствует букве "А", "Б" или "В" <br>
* `[А-ЯЁ]` - соответствует любой букве от "А" до "Я" <br>
* `[а-яА-ЯёЁ]` - соответствует любой русской букве в любом регистре <br>
* `[0-9а-яА-ЯёЁа-zА-Z]` - любая цифра и любая буква независимо от регистра и языка

Вместо перечисления символов можно использовать стандартные классы: <br>

* . - любой символ, кроме перевода строки (если точка не экранирована и не заключена в квадратные скобки)
* \d - соответствует любой цифре (эквивалентно [0-9]) <br>
* \w - соответствует любой букве, цифре или символу подчеркивания ([a-zA-Zа-яЁА-ЯЁ0-9_]) <br>
* \s - любой пробельный символ (пробел, перевод строки, табуляция и т.д.) <br>
* \D - не цифра (эквивалентно [^0-9]) <br>
* \W - не буква, не цифра и не символ подчеркивания (эквивалентно [^a-zA-Zа-яЁА-ЯЁ0-9_]) <br>
* \S - не пробельный символ <br>
* \b - обозначение левой или правой границы слова (где слово трактуется как последовательность букв или цифр)

С помощью квантификаторов задается количество вхождений символа в строку. Указывается после символа, к которому относится разрешенное количество повторений: <br>

* `{n}` - n вхождений символа в строку. Например. шаблон `r"[0-9]{2}"` соответствует двум вхождениям любой цифры
* `{n,}` - n или более вхождений символа в строку. Например. шаблон `r"[0-9]{2,}"` соответствует двум и более вхождениям любой цифры
* `{n,m}` - не менее n и не более m вхождений символа в строку. Числа указываются через запятую без пробела. 
    * Например, шаблон `r"[0-9]{2,4}"` соответствует от двух до четырех вхождениям любой цифры
* `*` - ноль или большее число вхождений символа в строку. Эквивалентно комбинации `{0,}`
* `+` - одно или большее число вхождений символа в строку. Эквивалентно комбинации `{1,}`
* `?` - ни одного или одно вхождение символа в строку. Эквивалентно комбинации `{0,1}`. 

Еще полезные символы:

* `^` - привязка к началу строки или подстроки.
* `$` - привязка к концу строки или подстроки.

Задача: сделайте шаблон для нахождения телефонов в формате +7 (xxx) xxx-xx-xx и найдите все такие телефоны в json'e `addres-book.json`, загрузив этот файл как строку